In [1]:
from selenium import webdriver
import time
import re

driver = webdriver.Chrome()
driver.get('https://personal.vanguard.com/us/FundsMFSBasicSearch?FROM=VAN')

time.sleep(3)

login = driver.find_element_by_css_selector("[value='0001-1']")
login.click()

time.sleep(3)

results = driver.find_element_by_css_selector("[value='Get Results']")
results.click()

time.sleep(3)

table = driver.find_element_by_css_selector(".dataTable")
data = table.text
item = data.split('\n')

# Risk Attributes
select_one = driver.find_element_by_css_selector("[value='RA']")
select_one.click()

table_one = driver.find_element_by_css_selector(".dataTable")
data_one = table_one.text
item_one = data_one.split('\n')

# Fund Preferences
select_two = driver.find_element_by_css_selector("[value='FP']")
select_two.click()

table_two = driver.find_element_by_css_selector(".dataTable")
data_two = table_two.text
item_two = data_two.split('\n')

# Fees and Minimums
select_three = driver.find_element_by_css_selector("[value='FM']")
select_three.click()

table_three = driver.find_element_by_css_selector(".dataTable")
data_three = table_three.text
item_three = data_three.split('\n')

# Management
select_four = driver.find_element_by_css_selector("[value='M']")
select_four.click()

table_four = driver.find_element_by_css_selector(".dataTable")
data_four = table_four.text
item_four = data_four.split('\n')

# Stock Characteristics
select_five = driver.find_element_by_css_selector("[value='EC']")
select_five.click()

table_five = driver.find_element_by_css_selector(".dataTable")
data_five = table_five.text
item_five = data_five.split('\n')

time.sleep(5)
driver.close()

In [2]:
# Performance (Part 1)
import numpy as np

def instance (old_item, document):
    indices = []
    k = 0
    while k < len(old_item)-2:
        if document in old_item[k+1]:
            indices.append(k+1)
        k = k + 1
    return indices

index = instance(item, 'Purchase Fee')
print(index)

def takeout (temp, num):
    print(temp[num])
    del temp[num]
    return temp

print(item[90])
print(item[94])
print(item[125])
print(item[132])

x = 1
while x <= len(index):
    print(index[-x])
    new = takeout(item, index[-x])
    x = x + 1

[90, 94, 125, 132]
Purchase Fee: 0.25
Purchase Fee: 0.25
Purchase Fee: 0.25
Purchase Fee: 0.25
132
Purchase Fee: 0.25
125
Purchase Fee: 0.25
94
Purchase Fee: 0.25
90
Purchase Fee: 0.25


In [3]:
# Performance (Part 2)
import numpy as np

index2 = instance(item, 'Redemption Fee')
    
def cleaner (old_item, document):
    indices = []
    k = 0
    while k < len(old_item)-1:
        if document in old_item[k+1]:
            temp = old_item[k+1].split('%')
            temp.remove(temp[0])
            
            new = ''
            for i in np.arange(len(temp)):
                new = new + temp[i]
            
            string = old_item[k] + new
            indices.append(string)
        k = k + 1
    return indices

value = cleaner(item, 'Redemption Fee')
print(value)

x = 1
while x <= len(index2):
    print(index2[-x])
    takeout(item, index2[-x])
    takeout(item, index2[-x]-1)
    x = x + 1

for i in np.arange(len(index)):
    item.insert(index[i]-1, value[i])

['Vanguard Global ex-U.S. Real Estate Index Fund Admiral Shares (VGRLX) Specialty Global Real Estate 0.12 —   — –12.03 3.58 — 4.05 02/10/2011 Invest now', 'Vanguard Global ex-U.S. Real Estate Index Fund Institutional Shares (VGRNX) Specialty Global Real Estate 0.11 —   — –12.02 3.59 — 3.89 04/19/2011 Invest now', 'Vanguard International Dividend Appreciation Index Fund Admiral Shares (VIAAX) International Stock Foreign Large Growth 0.20 — — 9.74 — — 9.28 03/02/2016 Invest now', 'Vanguard International High Dividend Yield Fund Admiral Shares (VIHAX) International Stock Foreign Large Value 0.27 — — –9.69 — — 3.77 03/02/2016 Invest now']
129
Redemption Fee: 0.25% International Stock Foreign Large Value 0.27% — — –9.69% — — 3.77% 03/02/2016 Invest now
Vanguard International High Dividend Yield Fund Admiral Shares (VIHAX)
123
Redemption Fee: 0.25% International Stock Foreign Large Growth 0.20% — — 9.74% — — 9.28% 03/02/2016 Invest now
Vanguard International Dividend Appreciation Index Fund 

In [4]:
# Performance (Part 3)

import numpy as np

def percent(old_percent):
    temp = []
    temp.extend(old_percent)
    temp.remove(temp[-1])
    
    new_string = ''
    for t in temp:
        new_string = new_string + t
    
    if new_string == '':
        new_percent = 0
    
    else:
        new_string = new_string.replace('\U00002013', '-')
        
        if new_string == 'not':
            new_string = '0'
        
        new_percent = float(new_string)
        new_percent = round(new_percent/100, 4)
    return new_percent

ls = ['fund_name', 'fund_type', 'fund_category', 'expense ratio', 'sec_yield', 'current_date', 'one_year',
     'five_year', 'ten_year', 'since inception', 'inception_date']

test = True
count = 0
for a in np.arange(19, 262):
    
    if test == False:
        test = True
    else:
        words = item[a].split(')')
        words[0] = words[0] + ')'
        ls.append(words[0])
        
        count = count + 1
        print(str(count) + ' ' + words[0])

        layer = words[1].split()

        if layer[0] == 'Specialty':
            ls.append(layer[0])
        else:
            ls.append(layer[0] + ' ' + layer[1])
            layer.remove(layer[1])

        layer.remove(layer[0])

        categories1 = ['Communications', 'Financial', 'Health', 'Industrials', 'Technology', 'Utilities'] 
        categories2 = ['Consumer', 'Equity', 'Europe', 'Large', 'Mid-Cap', 'Natural', 'Real', 'Small']

        switch = True

        if(switch):
            for cat1 in categories1:
                if layer[0] == cat1:
                    ls.append(layer[0])
                    layer.remove(layer[0])
                    switch = False

        if(switch):
            for cat2 in categories2:
                if layer[0] == cat2:
                    ls.append(layer[0] + ' ' + layer[1])
                    layer.remove(layer[1])
                    layer.remove(layer[0])
                    switch = False

        if(switch):
            ls.append(layer[0] + ' ' + layer[1] + ' ' + layer[2])
            layer.remove(layer[2])
            layer.remove(layer[1])
            layer.remove(layer[0])
            switch = False

        ls.append(percent(layer[0]))
        layer.remove(layer[0])

        ls.append(percent(layer[0]))
        layer.remove(layer[0])

        if layer[0] == 'B':
            layer.remove(layer[0])

        ls.append(layer[0])
        layer.remove(layer[0])

        for i in np.arange(4):
            ls.append(percent(layer[0]))
            layer.remove(layer[0])
        
        ls.append(layer[0])
        layer.remove(layer[0])

        if layer[0] == "Closed":
            test = False

1 Vanguard 500 Index Fund Admiral Shares (VFIAX)
2 Vanguard 500 Index Fund Investor Shares (VFINX)
3 Vanguard Capital Opportunity Fund Admiral Shares (VHCAX)
4 Vanguard Capital Opportunity Fund Investor Shares (VHCOX)
5 Vanguard Commodity Strategy Fund Admiral Shares (VCMDX)
6 Vanguard Communication Services Index Fund Admiral Class (VTCAX)
7 Vanguard Communication Services Index Fund ETF Shares (VOX)
8 Vanguard Consumer Discretionary Index Fund Admiral Shares (VCDAX)
9 Vanguard Consumer Discretionary Index Fund ETF Shares (VCR)
10 Vanguard Consumer Staples Index Fund Class Admiral Shares (VCSAX)
11 Vanguard Consumer Staples Index Fund ETF Shares (VDC)
12 Vanguard Developed Markets Index Fund Admiral Shares (VTMGX)
13 Vanguard Developed Markets Index Fund Institutional Plus Shares (VDIPX)
14 Vanguard Developed Markets Index Fund Institutional Shares (VTMNX)
15 Vanguard Developed Markets Index Fund Investor Shares (VDVIX)
16 Vanguard Diversified Equity Fund Investor Shares (VDEQX)
17 Va

In [5]:
# Risk Attributes
import numpy as np

def make_float (number):
    
    if number == '—':
        number = 0
    else:
        number = float(number)
    
    return number

ls_one = ['fund_name', 'beta', 'r-squared']

test = True
count = 0
for a in np.arange(12, 255):
    
    if test == False:
        test = True
    else:
        words_one = item_one[a].split(')')
        words_one[0] = words_one[0] + ')'
        ls_one.append(words_one[0])
        
        count = count + 1
        print(str(count) + ' ' + words_one[0])

        layer_one = words_one[1].split()
        
        ls_one.append(make_float(layer_one[0]))
        layer_one.remove(layer_one[0])
        
        ls_one.append(make_float(layer_one[0]))
        layer_one.remove(layer_one[0])

        if layer_one[0] == "Closed":
            test = False

1 Vanguard 500 Index Fund Admiral Shares (VFIAX)
2 Vanguard 500 Index Fund Investor Shares (VFINX)
3 Vanguard Capital Opportunity Fund Admiral Shares (VHCAX)
4 Vanguard Capital Opportunity Fund Investor Shares (VHCOX)
5 Vanguard Commodity Strategy Fund Admiral Shares (VCMDX)
6 Vanguard Communication Services Index Fund Admiral Class (VTCAX)
7 Vanguard Communication Services Index Fund ETF Shares (VOX)
8 Vanguard Consumer Discretionary Index Fund Admiral Shares (VCDAX)
9 Vanguard Consumer Discretionary Index Fund ETF Shares (VCR)
10 Vanguard Consumer Staples Index Fund Class Admiral Shares (VCSAX)
11 Vanguard Consumer Staples Index Fund ETF Shares (VDC)
12 Vanguard Developed Markets Index Fund Admiral Shares (VTMGX)
13 Vanguard Developed Markets Index Fund Institutional Plus Shares (VDIPX)
14 Vanguard Developed Markets Index Fund Institutional Shares (VTMNX)
15 Vanguard Developed Markets Index Fund Investor Shares (VDVIX)
16 Vanguard Diversified Equity Fund Investor Shares (VDEQX)
17 Va

In [6]:
def dollar_sign (number):
    broken = []
    broken.extend(number)
    
    old_list = []
    
    for b in np.arange(len(broken)):
        if broken[b] == '$':
            old_list.append(b)
        if broken[b] == ',':
            old_list.append(b)
    
    old_list.reverse()
    
    for old in old_list:
        del broken[old]
    
    put_back = ''
    for b in np.arange(len(broken)):
        put_back = put_back + broken[b]
    
    put_back = put_back.replace('\U00002013', '—')
    
    if put_back == '—':
        put_back = 0
    else:
        put_back == float(put_back)
    
    return put_back

In [7]:
# Fund Preferences

import numpy as np

def large_number(string):
    
    if 'b' in string:
        string = string.replace(' billion', '')
        string = dollar_sign(string)
        string = float(string)
        string = string * 1000000000
    
    elif 'm' in string:
        string = string.replace(' million', '')
        string = dollar_sign(string)
        string = float(string)
        string = string * 1000000
    
    return string

ls_two = ['fund_name', 'dollar_cost_averaging', 'fund_size']

test = True
count = 0
for a in np.arange(12, 255):
    
    if test == False:
        test = True
    else:
        words_two = item_two[a].split(')')
        words_two[0] = words_two[0] + ')'
        ls_two.append(words_two[0])
        
        count = count + 1
        print(str(count) + ' ' + words_two[0])

        layer_two = words_two[1].split()
        
        ls_two.append(layer_two[0])
        layer_two.remove(layer_two[0])
        
        ls_two.append(large_number(layer_two[0] + ' ' + layer_two[1]))
        layer_two.remove(layer_two[1])
        layer_two.remove(layer_two[0])

        if layer_two[0] == "Closed":
            test = False

1 Vanguard 500 Index Fund Admiral Shares (VFIAX)
2 Vanguard 500 Index Fund Investor Shares (VFINX)
3 Vanguard Capital Opportunity Fund Admiral Shares (VHCAX)
4 Vanguard Capital Opportunity Fund Investor Shares (VHCOX)
5 Vanguard Commodity Strategy Fund Admiral Shares (VCMDX)
6 Vanguard Communication Services Index Fund Admiral Class (VTCAX)
7 Vanguard Communication Services Index Fund ETF Shares (VOX)
8 Vanguard Consumer Discretionary Index Fund Admiral Shares (VCDAX)
9 Vanguard Consumer Discretionary Index Fund ETF Shares (VCR)
10 Vanguard Consumer Staples Index Fund Class Admiral Shares (VCSAX)
11 Vanguard Consumer Staples Index Fund ETF Shares (VDC)
12 Vanguard Developed Markets Index Fund Admiral Shares (VTMGX)
13 Vanguard Developed Markets Index Fund Institutional Plus Shares (VDIPX)
14 Vanguard Developed Markets Index Fund Institutional Shares (VTMNX)
15 Vanguard Developed Markets Index Fund Investor Shares (VDVIX)
16 Vanguard Diversified Equity Fund Investor Shares (VDEQX)
17 Va

In [8]:
# Fees and Minimums

import numpy as np

ls_three = ['fund_name', 'account_type', 'minimum_initial_investment']

test = True
count = 0
for a in np.arange(12, 255):
    
    if test == False:
        test = True
    else:
        words_three = item_three[a].split(')')
        words_three[0] = words_three[0] + ')'
        ls_three.append(words_three[0])
        
        count = count + 1
        print(str(count) + ' ' + words_three[0])

        layer_three = words_three[1].split()
        
        ls_three.append(layer_three[0])
        layer_three.remove(layer_three[0])
        
        ls_three.append(dollar_sign(layer_three[0]))
        layer_three.remove(layer_three[0])
        
        layer_three.remove(layer_three[0])
        layer_three.remove(layer_three[0])
        layer_three.remove(layer_three[0])
        layer_three.remove(layer_three[0])
        layer_three.remove(layer_three[0])

        if layer_three[0] == "Closed":
            test = False

1 Vanguard 500 Index Fund Admiral Shares (VFIAX)
2 Vanguard 500 Index Fund Investor Shares (VFINX)
3 Vanguard Capital Opportunity Fund Admiral Shares (VHCAX)
4 Vanguard Capital Opportunity Fund Investor Shares (VHCOX)
5 Vanguard Commodity Strategy Fund Admiral Shares (VCMDX)
6 Vanguard Communication Services Index Fund Admiral Class (VTCAX)
7 Vanguard Communication Services Index Fund ETF Shares (VOX)
8 Vanguard Consumer Discretionary Index Fund Admiral Shares (VCDAX)
9 Vanguard Consumer Discretionary Index Fund ETF Shares (VCR)
10 Vanguard Consumer Staples Index Fund Class Admiral Shares (VCSAX)
11 Vanguard Consumer Staples Index Fund ETF Shares (VDC)
12 Vanguard Developed Markets Index Fund Admiral Shares (VTMGX)
13 Vanguard Developed Markets Index Fund Institutional Plus Shares (VDIPX)
14 Vanguard Developed Markets Index Fund Institutional Shares (VTMNX)
15 Vanguard Developed Markets Index Fund Investor Shares (VDVIX)
16 Vanguard Diversified Equity Fund Investor Shares (VDEQX)
17 Va

In [9]:
# Management

import numpy as np

ls_four = ['fund_name', 'management_style', 'manager_name', 'management_tenure']

test = True
count = 0
for a in np.arange(12, 255):
    
    if test == False:
        test = True
    else:
        words_four = item_four[a].split(')')
        words_four[0] = words_four[0] + ')'
        ls_four.append(words_four[0])
        
        count = count + 1
        print(str(count) + ' ' + words_four[0])

        layer_four = words_four[1].split()
        
        if layer_four[0] == 'Actively':
            ls_four.append(layer_four[0] + ' ' + layer_four[1])
            layer_four.remove(layer_four[1])
            layer_four.remove(layer_four[0])
        else:
            ls_four.append(layer_four[0])
            layer_four.remove(layer_four[0])
        
        ls_four.append(layer_four[0])
        layer_four.remove(layer_four[0])
        
        ls_four.append(layer_four[0] + ' ' + layer_four[1])
        layer_four.remove(layer_four[1])
        layer_four.remove(layer_four[0])

        if layer_four[0] == "Closed":
            test = False

1 Vanguard 500 Index Fund Admiral Shares (VFIAX)
2 Vanguard 500 Index Fund Investor Shares (VFINX)
3 Vanguard Capital Opportunity Fund Admiral Shares (VHCAX)
4 Vanguard Capital Opportunity Fund Investor Shares (VHCOX)
5 Vanguard Commodity Strategy Fund Admiral Shares (VCMDX)
6 Vanguard Communication Services Index Fund Admiral Class (VTCAX)
7 Vanguard Communication Services Index Fund ETF Shares (VOX)
8 Vanguard Consumer Discretionary Index Fund Admiral Shares (VCDAX)
9 Vanguard Consumer Discretionary Index Fund ETF Shares (VCR)
10 Vanguard Consumer Staples Index Fund Class Admiral Shares (VCSAX)
11 Vanguard Consumer Staples Index Fund ETF Shares (VDC)
12 Vanguard Developed Markets Index Fund Admiral Shares (VTMGX)
13 Vanguard Developed Markets Index Fund Institutional Plus Shares (VDIPX)
14 Vanguard Developed Markets Index Fund Institutional Shares (VTMNX)
15 Vanguard Developed Markets Index Fund Investor Shares (VDVIX)
16 Vanguard Diversified Equity Fund Investor Shares (VDEQX)
17 Va

In [10]:
# Stock Characteristics

import numpy as np

def percent(old_percent):
    temp = []
    temp.extend(old_percent)
    temp.remove(temp[-1])
    
    new_string = ''
    for t in temp:
        new_string = new_string + t
    
    if new_string == '':
        new_percent = 0
    
    else:
        new_string = new_string.replace('\U00002013', '-')
        
        if new_string == 'not':
            new_string = '0'
        
        new_percent = float(new_string)
        new_percent = round(new_percent/100, 4)
    return new_percent

ls_five = ['fund_name', 'earnings_growth_rate', 'number_of_stocks', 'turnover_rate', 
           'price/earnings_ratio', 'price/book_ratio']

test = True
count = 0
for a in np.arange(12, 255):
    
    if test == False:
        test = True
    else:
        words_five = item_five[a].split(')')
        words_five[0] = words_five[0] + ')'
        ls_five.append(words_five[0])
        
        count = count + 1
        print(str(count) + ' ' + words_five[0])

        layer_five = words_five[1].split()
        
        ls_five.append(percent(layer_five[0]))
        layer_five.remove(layer_five[0])
        
        ls_five.append(int(layer_five[0]))
        layer_five.remove(layer_five[0])
        
        ls_five.append(percent(layer_five[0]))
        layer_five.remove(layer_five[0])
        
        ls_five.append(percent(layer_five[0])*100)
        layer_five.remove(layer_five[0])
        
        ls_five.append(percent(layer_five[0])*100)
        layer_five.remove(layer_five[0])

        if layer_five[0] == "Closed":
            test = False

1 Vanguard 500 Index Fund Admiral Shares (VFIAX)
2 Vanguard 500 Index Fund Investor Shares (VFINX)
3 Vanguard Capital Opportunity Fund Admiral Shares (VHCAX)
4 Vanguard Capital Opportunity Fund Investor Shares (VHCOX)
5 Vanguard Commodity Strategy Fund Admiral Shares (VCMDX)
6 Vanguard Communication Services Index Fund Admiral Class (VTCAX)
7 Vanguard Communication Services Index Fund ETF Shares (VOX)
8 Vanguard Consumer Discretionary Index Fund Admiral Shares (VCDAX)
9 Vanguard Consumer Discretionary Index Fund ETF Shares (VCR)
10 Vanguard Consumer Staples Index Fund Class Admiral Shares (VCSAX)
11 Vanguard Consumer Staples Index Fund ETF Shares (VDC)
12 Vanguard Developed Markets Index Fund Admiral Shares (VTMGX)
13 Vanguard Developed Markets Index Fund Institutional Plus Shares (VDIPX)
14 Vanguard Developed Markets Index Fund Institutional Shares (VTMNX)
15 Vanguard Developed Markets Index Fund Investor Shares (VDVIX)
16 Vanguard Diversified Equity Fund Investor Shares (VDEQX)
17 Va

In [11]:
import pandas as pd
import numpy as np

s = pd.Series(ls)
new_data = pd.DataFrame(s[0:len(s)].values.reshape(220, 11))
new_data.columns= s[0:11]

s_one = pd.Series(ls_one)
new_data_one = pd.DataFrame(s_one[0:len(s_one)].values.reshape(220, 3))
new_data_one.columns = s_one[0:3]

s_two = pd.Series(ls_two)
new_data_two = pd.DataFrame(s_two[0:len(s_two)].values.reshape(220, 3))
new_data_two.columns = s_two[0:3]

s_three = pd.Series(ls_three)
new_data_three = pd.DataFrame(s_three[0:len(s_three)].values.reshape(220, 3))
new_data_three.columns = s_three[0:3]

s_four = pd.Series(ls_four)
new_data_four = pd.DataFrame(s_four[0:len(s_four)].values.reshape(220, 4))
new_data_four.columns = s_four[0:4]

s_five = pd.Series(ls_five)
new_data_five = pd.DataFrame(s_five[0:len(s_five)].values.reshape(220, 6))
new_data_five.columns = s_five[0:6]

complete = new_data
complete = complete.join(new_data_one.set_index('fund_name'), on = 'fund_name')
complete = complete.join(new_data_two.set_index('fund_name'), on = 'fund_name')
complete = complete.join(new_data_three.set_index('fund_name'), on = 'fund_name')
complete = complete.join(new_data_four.set_index('fund_name'), on = 'fund_name')
complete = complete.join(new_data_five.set_index('fund_name'), on = 'fund_name')
complete

,fund_name,fund_type,fund_category,expense ratio,sec_yield,current_date,one_year,five_year,ten_year,since inception,...,account_type,minimum_initial_investment,management_style,manager_name,management_tenure,earnings_growth_rate,number_of_stocks,turnover_rate,price/earnings_ratio,price/book_ratio
0,fund_name,fund_type,fund_category,expense ratio,sec_yield,current_date,one_year,five_year,ten_year,since inception,...,account_type,minimum_initial_investment,management_style,manager_name,management_tenure,earnings_growth_rate,number_of_stocks,turnover_rate,price/earnings_ratio,price/book_ratio
1,Vanguard 500 Index Fund Admiral Shares (VFIAX),Domestic Stock,Large Blend,0.0004,0.0165,10/16/2020,0.1511,0.1411,0.1371,0.0675,...,Regular,3000,Index,Louie/Butler,4 years,0.2,508,0.04,25.9,3.6
2,Vanguard 500 Index Fund Investor Shares (VFINX),Domestic Stock,Large Blend,0.0014,0.0155,10/16/2020,0.15,0.14,0.1358,0.1108,...,Regular,0,Index,Louie/Butler,4 years,0.2,508,0.04,25.9,3.6
3,Vanguard Capital Opportunity Fund Admiral Shar...,Domestic Stock,Large Growth,0.0037,0.0043,10/16/2020,0.186,0.1465,0.1493,0.1107,...,Regular,0,Actively managed,Ansari/Marchetti/Mordecai/Kolokotrones/Fried,17 years,0.226,160,0.06,20.5,3.5
4,Vanguard Capital Opportunity Fund Investor Sha...,Domestic Stock,Large Growth,0.0044,0.0036,10/16/2020,0.1852,0.1457,0.1485,0.1235,...,Regular,0,Actively managed,Ansari/Marchetti/Mordecai/Kolokotrones/Fried,17 years,0.226,160,0.06,20.5,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,Vanguard Value Index Fund Investor Shares (VIVAX),Domestic Stock,Large Value,0.0017,0.0269,10/16/2020,-0.0343,0.0911,0.1065,0.0896,...,Regular,0,Index,Nejman/OReilly,15 years,0.152,333,0.12,18.7,2.2
216,Vanguard Windsor Fund Admiral Shares (VWNEX),Domestic Stock,Large Value,0.002,0.023,10/16/2020,-0.0334,0.0701,0.0998,0.0661,...,Regular,50000,Actively managed,Palmer/Silver/Flynn/Pzena,5 years,0.176,127,0.39,13.1,1.5
217,Vanguard Windsor Fund Investor Share (VWNDX),Domestic Stock,Large Value,0.003,0.022,10/16/2020,-0.0339,0.069,0.0987,0.1089,...,Regular,3000,Actively managed,Palmer/Silver/Flynn/Pzena,5 years,0.176,127,0.39,13.1,1.5
218,Vanguard Windsor II Fund Admiral Shares (VWNAX),Domestic Stock,Large Value,0.0026,0.0164,10/16/2020,0.0622,0.0972,0.1096,0.0678,...,Regular,50000,Actively managed,McBride/Temple/Padilla/Gleicher/Davis/Sanders,7 years,0.173,181,0.32,17.4,2.1


In [12]:
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

wb = Workbook()
ws = wb.active

writer = pd.ExcelWriter('vanguard.xlsx', engine = 'xlsxwriter')
complete.to_excel(writer, sheet_name = 'Sheet1')

writer.save()